In [6]:
"""
Cấu hình kết nối MongoDB và các thông số liên quan đến việc thu thập dữ liệu phim từ CGV.
"""

MONGO_URL = "mongodb+srv://CgvHub:o@cluster0.ktuns.mongodb.net/"
DATABASE_NAME = "cgv_movies_2"

# Tên collection trong MongoDB
FILM_COLLECTION_NAME = "films"
# URL của các trang phim
NOW_SHOWING_URL = "https://www.cgv.vn/default/movies/now-showing.html"
COMING_SOON_URL = "https://www.cgv.vn/default/movies/coming-soon-1.html"
# Thời gian chờ (giây) giữa các lần thu thập dữ liệu
WAIT_TIME = 2

In [46]:
from pymongo import MongoClient
import time
import logging
from typing import Any, Dict, List
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, WebDriverException
from selenium.webdriver.remote.webdriver import WebDriver
from pymongo.collection import Collection


In [47]:
def get_database():
    """
    Kết nối đến MongoDB và trả về đối tượng cơ sở dữ liệu.

    Returns:
        Database | None:
            - Đối tượng database nếu kết nối thành công.
            - None nếu có lỗi xảy ra.
    """
    try:
        client = MongoClient(MONGO_URL)
        database = client[DATABASE_NAME]
        print(f"Kết nối thành công đến cơ sở dữ liệu: '{DATABASE_NAME}'")
        return database
    except Exception as loi:
        print(f"Lỗi khi kết nối MongoDB: {loi}")
        return None


def save_to_database(col, data):
    """
    Lưu hoặc cập nhật thông tin phim vào cơ sở dữ liệu.

    Args:
        col (Collection): Collection phim trong MongoDB.
        data (dict): Dữ liệu phim cần lưu.

    Returns:
        None
    """
    result = col.update_one(
        {"title": data["title"]},
        {"$set": data},
        upsert=True
    )

    if result.matched_count == 0:
        print(f"Thêm mới: {data['title']}")
    else:
        print(f"Cập nhật: {data['title']}")



In [43]:
import time
from typing import Any, Dict, List
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from pymongo.collection import Collection

class Extractor:
    """
    Lớp trích xuất dữ liệu từ trang web bằng Selenium.
    """

    def __init__(self, driver: WebDriver, url: str):
        """
        Khởi tạo đối tượng Extractor.

        Args:
            driver (WebDriver): Trình điều khiển trình duyệt Selenium.
            url (str): URL của trang web cần trích xuất dữ liệu.
        """
        self.driver = driver
        self.driver.get(url)
        time.sleep(WAIT_TIME)

    def load_page(self, url: str):
        """
        Chuyển hướng trình duyệt đến URL mới.

        Args:
            url (str): URL của trang web mới.
        """
        self.driver.get(url)
        time.sleep(WAIT_TIME)

    def get_value_by_label(self, label: str) -> str:
        """
        Trả về giá trị văn bản của phần tử dựa trên nhãn được chỉ định.

        Args:
            label (str): Nhãn của phần tử cần lấy giá trị.

        Returns:
            str: Giá trị văn bản của phần tử hoặc None nếu không tìm thấy.
        """
        try:
            element = self.driver.find_element(
                By.XPATH, f"//label[contains(text(), '{label}')]/following-sibling::div"
            )
            return element.text.strip()
        except NoSuchElementException:
            return None

    def get_text_by_label(self, selector: str, attr: str = "text") -> str:
        """
        Trả về văn bản hoặc thuộc tính của phần tử nếu tìm thấy.

        Args:
            selector (str): Bộ chọn CSS của phần tử cần tìm.
            attr (str, optional): Thuộc tính cần lấy (mặc định là 'text').

        Returns:
            str: Văn bản hoặc giá trị thuộc tính của phần tử, hoặc None nếu không tìm thấy.
        """
        try:
            element = self.driver.find_element(By.CSS_SELECTOR, selector)
            return element.get_attribute(attr).strip() if attr != "text" else element.text.strip()
        except NoSuchElementException:
            return None

    def quit(self):
        """
        Đóng trình duyệt.
        """
        self.driver.quit()

    def extract_info(self):
        """
        Phương thức này sẽ được ghi đè trong các lớp con để trích xuất dữ liệu cụ thể.

        Returns:
            None
        """
        return None

    def is_data_available(self) -> bool:
        """
        Kiểm tra xem có thể lấy dữ liệu từ trình duyệt hay không. Nếu không, tự động làm mới trang.

        Returns:
            bool: True nếu dữ liệu có sẵn, False nếu không.
        """
        try:
            element = self.driver.find_element(By.TAG_NAME, "body")
            return bool(element)
        except NoSuchElementException:
            self.driver.refresh()
            time.sleep(WAIT_TIME)
            return False

    def upload_database(self, col: Collection):
        """
        Tải dữ liệu đã trích xuất lên cơ sở dữ liệu.

        Args:
            col (Collection): Bộ sưu tập MongoDB nơi lưu trữ dữ liệu.
        """
        if not self.is_data_available():
            time.sleep(WAIT_TIME)  # Chờ dữ liệu sẵn sàng

        data = self.extract_info()
        if data:
            save_to_database(col, data)

    def extract_list_film(self, col: Collection) -> List[Dict[str, Any]]:
        """
        Trích xuất danh sách phim từ trang web.

        Args:
            col (Collection): Bộ sưu tập MongoDB nơi lưu trữ dữ liệu.

        Returns:
            List[Dict[str, Any]]: Danh sách các phim trích xuất được.
        """
        return []


In [44]:
class CgvExtractor(Extractor):
    """
    Lớp con kế thừa từ Extractor để trích xuất thông tin phim.
    """

    def extract_list_film(self, col) -> List[Dict[str, Any]]:
        """
        Trích xuất danh sách phim từ trang web.

        Args:
            col: Đối tượng collection trong database để lưu thông tin phim.

        Returns:
            List[Dict[str, Any]]: Danh sách thông tin các bộ phim.
        """
        film_list = []
        try:
            film_links = [
                film.get_attribute("href")
                for film in self.driver.find_elements(By.CSS_SELECTOR, "a.product-image")
            ]

            for link_url in film_links:
                try:
                    self.load_page(link_url)
                    film_info = self.extract_info()
                    if film_info:
                        film_info["link"] = link_url
                        film_list.append(film_info)
                except WebDriverException as e:
                    logging.error(f"Lỗi Selenium khi xử lý phim {link_url}: {e}")
        except WebDriverException as e:
            logging.error(f"Lỗi khi tải trang: {e}")

        return film_list

    def extract_info(self) -> Dict[str, Any]:
        """
        Trích xuất thông tin chi tiết của một bộ phim từ trang web.

        Returns:
            Dict[str, Any]: Thông tin chi tiết của bộ phim.
        """
        film_info = {}
        try:
            # Lấy tiêu đề phim
            try:
                title_element = self.driver.find_element(
                    By.CSS_SELECTOR, "div.product-name span.h1"
                )
                film_info["title"] = title_element.text.strip()
            except Exception:
                film_info["title"] = "Không có thông tin"

            # Lấy các thông tin khác
            film_info["genre"] = self.get_value_by_label("Thể loại")
            film_info["duration"] = self.get_value_by_label("Thời lượng")
            film_info["release_date"] = self.get_value_by_label("Khởi chiếu")
            film_info["director"] = self.get_value_by_label("Đạo diễn")
            film_info["language"] = self.get_value_by_label("Ngôn ngữ")
            film_info["rated"] = self.get_value_by_label("Rated")

            # Lấy thông tin diễn viên
            actors_text = (
                self.get_value_by_label("Diễn viên")
                or self.get_value_by_label("Diễn viên chính")
                or "Không có thông tin"
            )
            film_info["actors"] = actors_text

            # Lấy mô tả phim
            try:
                description_element = self.driver.find_element(
                    By.CSS_SELECTOR, "meta[name='description']"
                )
                film_info["description"] = description_element.get_attribute("content").strip()
            except Exception:
                film_info["description"] = None

            # Lấy các định dạng công nghệ phim
            try:
                technology_elements = self.driver.find_elements(
                    By.CSS_SELECTOR, "div.movie-technology-icons a.movie-detail-icon-type span"
                )
                film_info["technologies"] = [
                    tech.text.strip() for tech in technology_elements if tech.text.strip()
                ]
            except Exception:
                film_info["technologies"] = []

            # Lấy poster phim
            try:
                poster_element = self.driver.find_element(
                    By.CSS_SELECTOR, "img#image-main.gallery-image.visible"
                )
                film_info["poster"] = poster_element.get_attribute("src")
            except Exception:
                film_info["poster"] = None

            # Kiểm tra vé có sẵn không
            film_info["ticket_available"] = bool(
                self.driver.find_elements(By.CSS_SELECTOR, "button.btn-booking")
            )
        except Exception as e:
            logging.error(f"Lỗi khi lấy thông tin phim: {e}")
            return {}

        return film_info


In [35]:
service = Service()
driver = webdriver.Chrome(service=service)
extractor = CgvExtractor(driver, "https://www.cgv.vn/default/quy-nhap-trang.html")
film = extractor.extract_info()
print(film)
extractor.quit()

{'title': 'QUỶ NHẬP TRÀNG', 'genre': 'Kinh Dị', 'duration': '121 minutes 51 seconds', 'release_date': '07/03/2025', 'director': 'Pom Nguyễn', 'language': 'Tiếng Việt và phụ đề tiếng Anh', 'rated': 'T18 - PHIM ĐƯỢC PHỔ BIẾN ĐẾN NGƯỜI XEM TỪ ĐỦ 18 TUỔI TRỞ LÊN (18+)', 'actors': 'Quang Tuấn, Khả Như, NSƯT Phú Đôn, Vân Dung, NSND Thanh Nam, Hoàng Mèo, Thanh Tân, Trung Ruồi, Kiều Chi,…', 'description': 'Lấy cảm hứng từ truyền thuyết kinh dị nhất về “người chết sống dậy”, Quỷ Nhập Tràng là câu chuyện được lấy bối cảnh tại một ngôi làng chuyên nghề mai táng, gắn liền với những hoạt động đào mộ, tẩm liệm và chôn cất người chết.', 'technologies': ['4DX', 'Starium'], 'poster': 'https://iguov8nhvyobj.vcdn.cloud/media/catalog/product/cache/1/image/c5f0a1eff4c394a251036189ccddaacd/q/u/qu_nh_p_tr_ng_-_payoff_poster_-_kc_07032025_1_.jpg', 'ticket_available': True}


In [45]:
service = Service()
driver = webdriver.Chrome(service=service)
extractor = CgvExtractor(driver, NOW_SHOWING_URL)
# Kết nối MongoDB
db = get_database()
if db is None:
    print("Kết nối cơ sở dữ liệu thất bại. Đang thoát...")

films_col = db[FILM_COLLECTION_NAME]
films = extractor.extract_list_film(films_col )
for film in films:
    print(film)
extractor.quit()

Kết nối thành công đến cơ sở dữ liệu: 'cgv_movies_2'


ServerSelectionTimeoutError: SSL handshake failed: cluster0-shard-00-02.ktuns.mongodb.net:27017: [SSL: TLSV1_ALERT_INTERNAL_ERROR] tlsv1 alert internal error (_ssl.c:1006) (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),SSL handshake failed: cluster0-shard-00-01.ktuns.mongodb.net:27017: [SSL: TLSV1_ALERT_INTERNAL_ERROR] tlsv1 alert internal error (_ssl.c:1006) (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),SSL handshake failed: cluster0-shard-00-00.ktuns.mongodb.net:27017: [SSL: TLSV1_ALERT_INTERNAL_ERROR] tlsv1 alert internal error (_ssl.c:1006) (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 67e4b409cd190d4757e6e2ee, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('cluster0-shard-00-00.ktuns.mongodb.net', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('SSL handshake failed: cluster0-shard-00-00.ktuns